In [1]:
!pip freeze

accelerate==0.27.2
aiohttp==3.9.1
aiosignal==1.3.1
annotated-types==0.6.0
annoy==1.17.3
antlr4-python3-runtime==4.9.3
anyio @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_a17a7759g2/croot/anyio_1706220182417/work
appdirs==1.4.4
appnope @ file:///home/conda/feedstock_root/build_artifacts/appnope_1707233003401/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///Users/cbousseau/work/recipes/ci_py311/argon2-cffi-bindings_1677915727169/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1698341106958/work
async-lru @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_02efro5ps8/croot/async-lru_1699554529181/work
attrs @ file:///private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_224434dqzl/croot/attrs_1695717839274/work
Babel @ file:///Users/cbousseau/work/recipes/ci_py311/babel_1677920677615/work
backoff==2.2.1
beautifulsoup4 @ file:///private/var/folders/nz/j6p

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
# %cd drive/MyDrive/ANLP/NLP-RAG/src-rag
# !ls

In [4]:
import os
# from dotenv import load_dotenv
# load_dotenv('.env')
# hf_api = os.getenv('HF_API')

In [5]:
pip install transformers==4.38.0

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install -q torch accelerate bitsandbytes langchain sentence-transformers faiss-cpu openpyxl
# !pip install -q torch accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl

In [10]:
# !pip install unstructured ragatouille
# # reranker
# from ragatouille import RAGPretrainedModel

In [11]:
# # this will take time :c
# !CUDACXX=/usr/local/cuda-12/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir --force-reinstall --upgrade

In [12]:
# !pip install ctransformers[cuda]

In [7]:
!pip3 install numpy==1.26.4
import numpy as np
np.__version__

'1.26.4'

In [8]:
# fix colab error: https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.docstore.document import Document as LangchainDocument
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate, ChatPromptTemplate
import os
import pandas as pd

In [10]:
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [11]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
pd.set_option(
    "display.max_colwidth", None
)

In [12]:
# pip list --outdated

# Specify the model/versions, etc.
## also specify the knowledge base file and the reference answer directory

In [13]:
# QUESTIONS_FILE = '../data/test/questions.txt' # change when we have all the test ones / for the category wise performance analysis
EMBEDDING_MODEL = "thenlper/gte-base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
# RERANKER = RAGPretrainedModel.from_pretrained(RERANKER_MODEL)

In [14]:
EMBEDDING_MODEL_NAME = EMBEDDING_MODEL
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    # model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

# Running on processed schedule files

In [15]:
from langchain_community.document_loaders import TextLoader, CSVLoader
import glob
from database import split_documents, create_db
directory_csv = '../data/schedule_csv/'
csv_files = glob.glob(os.path.join(directory_csv, '**', '*.csv'), recursive=True)

In [16]:
len(csv_files)

4

In [17]:
loader_csv = DirectoryLoader(directory_csv, glob="**/*.csv",
                             use_multithreading = True,
                             loader_cls=CSVLoader,
                            #  loader_kwargs = csv_args
                             )
docs_csv = loader_csv.load()
docs_csv

[Document(page_content='Question: What is the course number of IDeATe: Introduction to 3D Animation Pipeline?\nAnswer: 60125', metadata={'source': '../data/schedule_csv/sched_summer24_1_nl.csv', 'row': 0}),
 Document(page_content='Question: What is the name of course number 60125?\nAnswer: IDeATe: Introduction to 3D Animation Pipeline', metadata={'source': '../data/schedule_csv/sched_summer24_1_nl.csv', 'row': 1}),
 Document(page_content='Question: What time does the course 60125 IDeATe: Introduction to 3D Animation Pipeline begin?\nAnswer: 12:30PM', metadata={'source': '../data/schedule_csv/sched_summer24_1_nl.csv', 'row': 2}),
 Document(page_content='Question: What time does the course 60125 IDeATe: Introduction to 3D Animation Pipeline end?\nAnswer: 01:50PM', metadata={'source': '../data/schedule_csv/sched_summer24_1_nl.csv', 'row': 3}),
 Document(page_content='Question: What days does the course 60125 IDeATe: Introduction to 3D Animation Pipeline meet?\nAnswer: Monday, Tuesday, Wed

In [19]:
db_csv = create_db(docs_csv)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [25]:
db_csv.save_local("../faiss_index_schedule_csv")